# DCA Strategies Program

## Synopsis
* This program is to see what DCA (dollar cost averaging) investing frequency yields the greatest return on various stocks.

## How to use this program
### DCA strategy
* Remove the "#" in front of python```!pip install yfinance``` in the next cell and begin running the cells
* in order to generate the data for a specific stock, define a variable using the python```getMaxData() function```
* Enter the exact ticker name in quotations into the python```getMaxData()``` function (search up on yahoo finance if needed)
* To analyze the data, call the python```DCA()``` function. Enter the data from the previously defined variable. There are also options to define parameters for the interval and investment.
* The interval parameter has options for "D" for daily investment,  "W" for weekly investments, "2W" for biweekly investments, and "M" for monthly investments.
* The investment parameter is the total annual investment you would like to make.

### Sept investments
* Historically september has always been the lowest month of the stock market. there is a function to calculate what you would've returned if you made 4 equal investments during the month of september each year. 
* to use this feature, first initialize a variable example python```vfv_sept = gette("VFV.TO")```, then plug it into the function python```septInvestment(vfv_sept, "VFV.TO", 24000)```. 
* This function accepts 3 parameters, the first is the array initialized previously, the 2nd is the ticker name, and the third optional variable is the annual investment you want to make. By default it is $24000. 

### Get all Results
* A convenient feature to get all the results for all DCA strategies and september-only investing. just call the function python```getAllResults()```. It has 2 parameters, one is the exact ticker name from yahoo finance, and the other is the annual investment amount which by default is set to $24000.

- Please note that there are exactly 12 equal monthly investments, 26 equal biweekly investments, 52 equal weekly investments, and 252 equal daily investments as the stock market has 252 trading days.
- By default, the parameters for the DCA function are "D" and 24000 annual investment. 

In [1]:
#!pip install yfinance

In [1]:
import yfinance as yf
import pandas as pd
import multiprocessing
from datetime import datetime, timedelta

In [ ]:
def refactor_table(array):
    """
    this function turns the dataframe table into a 2D array with the date and price
    """
    for i in range(len(array)):
        date_str = str(array[i][0])[:10]  # Format date as 'YYYY-MM-DD'
        price = array[i][1]
        array[i] = [date_str, price]      # Replace entire row
    return array

In [3]:
def getMaxData(ticker):
    """
    this function downloads the ticker data from yahoo finance. You can adjust the period to specific timeframes as well. 
    """
    ticker_dataframe = yf.download(ticker, period="max")['Close']
    ticker_array = ticker_dataframe.reset_index().to_numpy()
    return refactor_table(ticker_array)


In [4]:
def get_interval_prices(ticker, interval):
    """
    this function returns a list of the prices for weekly and biweekly intervals. 
    """
    prices = []
    last_date = None

    for date_str, price in ticker:
        current_date = datetime.strptime(date_str, "%Y-%m-%d")

        if last_date is None or (current_date - last_date).days >= interval:
            prices.append(price)
            last_date = current_date
    return prices

In [5]:
def DCA(ticker, interval='D', investment=24000):
    """
    interval options: 'D', 'W', '2W', 'M'
    """
    results = {}
    prices = []
    stock_count = 0

    # generates the prices table and defines interval investment variables
    if interval == 'D':
        for i in range(len(ticker)):
            prices.append(ticker[i][1])
        interval_investment = investment / 252
    elif interval == 'W':
        for i in range(0, len(ticker), 7):
            prices = get_interval_prices(ticker, 7)
        interval_investment = investment / 52
    elif interval == '2W':
        for i in range(0, len(ticker), 14):
            prices = get_interval_prices(ticker, 14)
        interval_investment = investment / 26
    else:
        seen_months = []
        for i in range(len(ticker)):
            year_month = ticker[i][0][0:7]

            if year_month not in seen_months:
                prices.append(ticker[i][1])
                seen_months.append(year_month)
        interval_investment = investment / 12

    # calculates returns and portfolio value
    for i in range(len(prices)):
        stock_count += interval_investment / prices[i]

    portfolio_value = stock_count * prices[-1]
    portfolio_return = (portfolio_value / investment - 1) * 100

    # returns results
    results =  {
        "annual investment": investment, 
        "interval": interval,
        "interval investment": f"${round(interval_investment, 2)}",
        "portfolio value": f"${round(portfolio_value, 2)}", 
        "portfolio return": f"{round(portfolio_return, 2)}%", 
    }

    for k, v in results.items():
        print(f"{k}: {v}")
    

In [21]:
def getSeptData(ticker):
    """
    this function returns the data for the 4 weeks of September for a given ticker
    """
    ticker_dataframe = yf.download(ticker, period="max")['Close']
    ticker_array = ticker_dataframe.reset_index().to_numpy()
    return refactorSept(ticker_array)

In [26]:
def refactorSept(array):
    """
    this function refactors the array to only include the 4 weeks of September
    """
    from collections import defaultdict
    weekly_data = defaultdict(lambda: [None, None, None, None])

    for date_str, price in array:
        date_str = str(date_str)[:10]  # Format date as 'YYYY-MM-DD'
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")

        if date_obj.month == 9:
            year = date_obj.year
            day = date_obj.day

            if 1 <= day <= 7:
                week = 0
            elif 8 <= day <= 14:
                week = 1
            elif 15 <= day <= 21:
                week = 2
            elif 22 <= day <= 28:
                week = 3
            else:
                continue

            if weekly_data[year][week] is None:
                weekly_data[year][week] = price
    return [weekly_data[year] for year in sorted(weekly_data)]

In [55]:
def getRecentPrice(ticker):
    """
    this function returns the most recent price of a ticker
    """
    ticker_dataframe = yf.download(ticker, period="1d")['Close']
    ticker_array = ticker_dataframe.reset_index().to_numpy()
    return ticker_array[-1][1] if ticker_array.size > 0 else None

In [60]:
def septInvestment(ticker_array, ticker, investment=24000):
    """
    this function calculates the returns for a DCA strategy in September
    """
    
    interval_investment = investment / 4  # Assuming 4 weeks in September
    stock_count = 0

    for year in range(len(ticker_array)):
        for week in range(len(ticker_array[year])):
            week_price = ticker_array[year][week]
            if week_price is not None:
                stock_count += interval_investment / week_price

    portfolio_value = stock_count * getRecentPrice(ticker)  # Last week's price
    portfolio_return = (portfolio_value / investment - 1) * 100

    results =  {
        "annual investment": investment, 
        "interval investment": f"${round(interval_investment, 2)}",
        "portfolio value": f"${round(portfolio_value, 2)}", 
        "portfolio return": f"{round(portfolio_return, 2)}%", 
    }

    for k, v in results.items():
        print(f"{k}: {v}")


In [80]:
def getAllResults(ticker, investment=24000):
    """
    this function returns all results for a given ticker
    """
    ticker_data = getMaxData(ticker)
    sept_data = getSeptData(ticker)
    
    print(f"Results for {ticker}:")
    print("-------------DAILY--------------")
    DCA(ticker_data, 'D', investment)
    print("-------------WEEKLY-------------")
    DCA(ticker_data, 'W', investment)
    print("-------------BIWEEKLY-----------")
    DCA(ticker_data, '2W', investment)
    print("-------------MONTHLY------------")
    DCA(ticker_data, 'M', investment)
    print("-------------SEPT ONLY----------")
    septInvestment(sept_data, ticker, investment)

In [81]:
getAllResults("VFV.TO", 24000)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Results for VFV.TO:
-------------DAILY--------------
annual investment: 24000
interval: D
interval investment: $95.24
portfolio value: $856865.01
portfolio return: 3470.27%
-------------WEEKLY-------------
annual investment: 24000
interval: W
interval investment: $461.54
portfolio value: $861846.35
portfolio return: 3491.03%
-------------BIWEEKLY-----------
annual investment: 24000
interval: 2W
interval investment: $923.08
portfolio value: $853186.96
portfolio return: 3454.95%
-------------MONTHLY------------
annual investment: 24000
interval: M
interval investment: $2000.0
portfolio value: $860701.57
portfolio return: 3486.26%
-------------SEPT ONLY----------


[*********************100%***********************]  1 of 1 completed

annual investment: 24000
interval investment: $6000.0
portfolio value: $802301.88
portfolio return: 3242.92%
